# Deep Neural Network(DNN) for data fitting
### Shuhe Wang, Dec 2019

## How to install

#### STEP1:
pip install --upgrade pip
#### STEP2:
sudo pip install --no-cache-dir -I tensorflow==1.7     (If you have tensorflow installed)

sudo pip install tensorflow==1.7                        (If you haven't installed it yet.)

## Import packages and functions

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.contrib.framework.python.framework import checkpoint_utils as cu
from random import shuffle
import numpy as np
import os

print(tf.__version__)

## Customize global parameters

In [ ]:
# Customize global parameters here.
"""
    set size of data examples, how they are allocated to different sets and the training loop size
"""
### START CODE HERE ### 
num_examples = None
validation_fraction = None
test_fraction = None
training_epochs = None
### START CODE HERE ### 


PLOT_FIGURE = True
figure_size = (16.0, 9.0)
marker_size = 5.0
neuron_of_layer = [1, 50, 50, 50, 50, 50, 50, 50, 1]
beta = 0.01

# The parameters below are automatically generated
# !!! DO NOT CHANGE THEM !!!
validation_set_size = int(num_examples*validation_fraction)
test_set_size = int(num_examples*test_fraction)
training_set_size = num_examples - validation_set_size - test_set_size
training_epochs_digits = len(list(str(training_epochs)))
display_step = int(training_epochs*0.1)
num_of_all_layers = len(neuron_of_layer)
num_of_hidden_layers = num_of_all_layers - 2

assert type(num_examples)==int and num_examples>0
assert training_set_size>0
assert type(training_epochs)==int and training_epochs>10
for i in neuron_of_layer:
	assert type(i)==int and i>0

## Generate data sets

In [ ]:
def f(x: np.array) -> np.array:
    """
        DEFINE YOUR PREFERRED FUNCTIONS HERE (e.g. return np.cos(x))
        MAKE SURE YOUR FUNCTION VALUE IS NOT SO LARGE FOR X IN (0, PI)
    """
    ### START CODE HERE ### 
    pass
    ### END CODE HERE ### 

In [ ]:
all_x = np.random.uniform(0, np.pi, (1, num_examples)).T
np.random.shuffle(all_x)

"""
    USE SLICING TO PARTITION SETS
    YOU MAY USE VARIABLES : training_set_size, validation_set_size, test_set_size
"""
### START CODE HERE ### 
x_training, x_validation, x_test = (all_x[:None], 
                                    all_x[None: None],
                                    all_x[None:])
### START CODE HERE ### 

"""
    USE f(X) TO GENERATE Y 
"""
### START CODE HERE ### 
(y_training, y_validation, y_test) = (None)
### END CODE HERE ### 

In [ ]:
try:
    os.mkdir("./figure")
except:
    pass

figure_path = os.getcwd() + '/figure'

if PLOT_FIGURE:
    fig = plt.figure(figsize=figure_size)
    plt.scatter(x_training, y_training, c='r', label='training set', s=marker_size)
    plt.scatter(x_validation, y_validation, c='b', label='validation set', s=marker_size)
    plt.legend()
    fig.savefig(os.path.join(figure_path, 'training_and_validation_sets.jpg'))

## Set DNN training parameters 

In [ ]:
"""
    RESET THE TF NETWORK GRAPH BEFORE INITIALIZATION 
"""
### START CODE HERE ### 

### END CODE HERE ### 

### START CODE HERE ### 
(X, Y) = (tf.placeholder(tf.float32, [None, None], name='Input'),
          tf.placeholder(tf.float32, [None, None], name='Output'))
### END CODE HERE ###                         

"""
    CREATE WEIGHTS AND BIASES
    USE: tf.Variable(), tf.random_normal(), tf.zeros()
"""
### START CODE HERE ### 
(weights, biases) = ({str(i): tf.Variable(None,
                          name = 'w{:d}'.format(i)) for i in range(num_of_all_layers-1)},
                     {str(i): tf.Variable(None, 
                          name = 'b{:d}'.format(i)) for i in range(num_of_all_layers-1)})
### END CODE HERE ###  

In [ ]:
"""
    REPLACE 'NONE' WITH TF FUNCTIONS
    USE: tf.add(), tf.matmul(), tf.nn.sigmoid()
"""
### START CODE HERE ### 
def DNN(input_data, num_of_hidden_layers):
    def _build_hidden(num_of_hidden_layers):
        current_layer = 0
        output_current_layer = input_data[:]
        while current_layer < num_of_hidden_layers:
            Z = None(
                    None(output_current_layer, weights['%d'%(current_layer)]),
                    biases['%d'%(current_layer)]
                )
            output_current_layer = None(Z, name='layer%d'%(current_layer))
            current_layer += 1
        return str(current_layer), output_current_layer 

    next_layer, layer = _build_hidden(num_of_hidden_layers)  
    output_NN = None(None(layer, weights[next_layer]), biases[next_layer])
    return output_NN
### END CODE HERE ###  

## Set up loss and add regularization

In [ ]:
output = DNN(X, num_of_hidden_layers)
cost = tf.reduce_mean(tf.square(output - Y))
loss = tf.nn.l2_loss(output - Y)
for key in weights:
    loss += tf.nn.l2_loss(weights[key]) * beta
for key in biases:
    loss += tf.nn.l2_loss(biases[key]) * beta
optimizer = tf.train.AdamOptimizer().minimize(loss)

## Initialize DNN

In [ ]:
"""
    INITIALIZE
    USE: tf.InteractiveSession(), tf.global_variables_initializer(), sess.run()
"""
### START CODE HERE ### 
sess = None
init = None
None
### END CODE HERE ###  

## Train the DNN

In [ ]:
for epoch in range(1, training_epochs+1):
    _, c = sess.run(
        [optimizer, cost ],
        feed_dict={X: x_training, Y: y_training}
    )

    mse = sess.run(tf.nn.l2_loss(output - y_validation), feed_dict={X: x_validation})
    
    if not epoch % display_step :
        print('Epoch: %0*d\tCost : %.6f'%(training_epochs_digits, epoch, c))
        if PLOT_FIGURE:
            fig = plt.figure(figsize=figure_size)
            plt.scatter(x_validation, y_validation, color='b', label='validation set', s=marker_size)
            plt.scatter(x_validation, sess.run(output, feed_dict={X: x_validation}),
                        color='r', label='DNN prediction', s=marker_size) 
            plt.legend()         
            fig.savefig(os.path.join(figure_path, '%*d_DNN_prediction.jpg'%(training_epochs_digits, epoch)))

print('Epoch: %03d\tMSE  : %.6f'%(epoch, c), 'Training complete!', sep='\n')

save_model_list = [weights[i] for i in weights] + [biases[i] for i in biases]
saver = tf.train.Saver(save_model_list)
saver.save(sess, './trained_model/trained_model.ckpt')

plt.close('all')
sess.close()

## Run your trained DNN now

In [ ]:
def sigmoid(x):
    """
        DEFINE THE SIGMOID FUNCTION HERE
        use np.exp() instead of math.exp() since x is a numpy array
    """
    ### START CODE HERE ### 
    pass
    ### END CODE HERE ### 


def _DNN(input_data, num_of_hidden_layers):
    def _build_hidden(num_of_hidden_layers):
        current_layer = 0
        output_current_layer = input_data[:]
        while current_layer < num_of_hidden_layers:
            Z = np.add(
                    np.matmul(output_current_layer, _weights['%d'%(current_layer)]),
                    _biases['%d'%(current_layer)]
                )
            output_current_layer = sigmoid(Z)
            current_layer += 1
        return str(current_layer), output_current_layer 
        
    next_layer, layer = _build_hidden(num_of_hidden_layers)  
    output_NN = np.add(np.matmul(layer, _weights[next_layer]), _biases[next_layer])
    return output_NN

In [ ]:
_ckpt_reader = tf.train.load_checkpoint('./trained_model/')
for model_element in cu.list_variables('./trained_model/'):
    print(model_element)

In [ ]:
(_weights, _biases) = ({str(i): _ckpt_reader.get_tensor('w{:d}'.format(i)) for i in range(num_of_all_layers - 1)}, 
                       {str(i): _ckpt_reader.get_tensor('b{:d}'.format(i)) for i in range(num_of_all_layers - 1)})

_output = _DNN(x_test, num_of_hidden_layers)

In [ ]:
_fig = plt.figure(figsize = figure_size)
plt.scatter(x_test, y_test, color='r', label='test set', s=marker_size)
plt.scatter(x_test, _output, color='g', label='DNN prediction', s=marker_size)
plt.legend()
plt.show()

In [ ]:
sess.close()
plt.close()

## Congratulations! You can uninstall TF now!

#### STEP3:
sudo pip uninstall tensorflow 